### Liczba klastrów oraz metryki jakości klasteryzacji.

Weźmy pierwszy przykład.

In [ ]:
import pandas as pd
from sklearn.datasets import make_blobs

data = make_blobs(n_samples=1000, centers=2, cluster_std=1.0, center_box=(-4.0, 4.0), random_state=42)[0]
df = pd.DataFrame(data, columns=['x1', 'x2'])
df.head()

In [ ]:
import plotly.express as px

px.scatter(df, 'x1', 'x2', width=950, height=500, title='Dane')

I popatrzmy jak się zachowa dla różnej liczby klastrów.

Zacznijmy najbardziej naturalnie. Od dwóch klastrów.

In [ ]:
from sklearn.cluster import KMeans

# na początek dwa klastrów
kmeans = KMeans(n_clusters=2)
kmeans.fit(data)

In [ ]:
y_kmeans = kmeans.predict(data)
df['y_kmeans'] = y_kmeans
df.head()

Narysujmy to

In [ ]:
import plotly.graph_objects as go

centroid_1, centroid_2 = kmeans.cluster_centers_

fig = px.scatter(df, 'x1', 'x2', 'y_kmeans', width=950, height=500, title='Algorytm K-średnich - 2 klastry')
fig.add_trace(go.Scatter(x=[centroid_1[0]], y=[centroid_1[1]], name='centroid 1', mode='markers', marker_line_width=2))
fig.add_trace(go.Scatter(x=[centroid_2[0]], y=[centroid_2[1]], name='centroid 2', mode='markers', marker_line_width=2))
fig.update_layout(showlegend=False)
fig

In [ ]:
cluster_1 = df[df.y_kmeans==0]
cluster_2 = df[df.y_kmeans==1]

Napiszmy funkcję, która będzie liczyła miarę wss dla zadanego klastra (przy założeniu, że pracujemy z dwoma zmiennymi 'x1', 'x2').

In [ ]:
import numpy as np
from numpy.linalg import norm

def wss(cluster: pd.DataFrame) -> float:
    """Calculates the sum of distances between centroid of the given cluster and every point in this cluster."""
    
    centroid = np.array([cluster.x1.mean(), cluster.x2.mean()])
    
    distance = 0
    for index, row in cluster.iterrows():
        point = np.array([row.x1, row.x2])
        distance += norm(centroid - point)**2  # kwadratów odległości (kwadrat normy wektora różnicy)
    return distance

Teraz policzmy wss dla obu klastrów.

In [ ]:
wss_1 = wss(cluster_1)
wss_2 = wss(cluster_2)

print(wss_1)
print(wss_2)

Policzmy całkowitą dyspersję

In [ ]:
total_wss = wss_1+wss_2
print(total_wss)

Teraz należałoby policzyć wss dla jednego klastra, trzech, czterech, ... I porównać je. Trochę roboty.

Na szczęscie nie musimy tego ręcznie liczyć. Obiekt klasy KMeans przechowuje w swoim atrybucie 'intertia_' informacje o całkowitej wartości metryki wss.

In [ ]:
kmeans.inertia_  # pojawiają się jakieś drobne niedokładności wynikłe z artytmetyki zmiennoprzecinkowej

Zróbmy zestawienie metryki **wss** dla różnej liczny klastrów wykorzystując atrybut **inertia_**